In [13]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# 切り上げ
import math

# スクレイピング
import requests
from bs4 import BeautifulSoup

In [3]:
url = "https://seesaawiki.jp/hololivetv/d/%c0%b1%b3%b9%a4%b9%a4%a4%a4%bb%a4%a4%a1%da%b2%ce%be%a7%b3%da%b6%ca%b0%ec%cd%f7%a1%db"

r = requests.get(url)

# 文字化けが発生しているので予めデコードしてから利用
soup = BeautifulSoup(r.content.decode("euc-jp", "ignore"), "html.parser")

# 取得できているか確認
print(soup.select("h1"))

[<h1><a href="https://seesaawiki.jp/hololivetv/">ホロライブ非公式wiki</a></h1>]


In [5]:
# 歌枠全体を管理しているテーブル
# 枠そのものはダブルチェックくらいにしか使わないので一旦保留
streams = soup.find_all("table", {"id":"content_block_13"})[0]
rows = streams.find_all("tr")

In [6]:
# あれ、カリとのライブがない？？？
# 足りない情報がないか確認する手段が欲しい

# 楽曲を管理しているテーブル
songs = soup.find_all("table", {"id":"content_block_19"})[0]
rows = songs.find_all("tr")

# リンクがある曲だけ取得する
def check_cell(cells):
    # リンクそのものが存在しない場合はfalse
    if cells[2].a == None:
        return False
    
    href = cells[2].a["href"]
    # 短縮系もあるため"youtu"で判別
    if "youtu" not in href:
        return False

    # 備考に非公開とある場合false
    text = cells[3].text
    if "非公開" in text:
        return False
    return True

# リンクから動画IDと開始時間を取得する
def get_videoid_time(cells):
    href = cells[2].a["href"]
    if "youtube" in href:
        stratid = href.find("v=")+2
        endid = href.find("&t=")
        videoid = href[stratid:endid]
        time = href[endid+3:-1]
    if "youtu.be" in  href:
        stratid = href.find(".be")+4
        endid = href.find("?t=")
        videoid = href[stratid:endid]
        time = href[endid+3:]
    return videoid, time
    
# メンバーと備考からアーティストを取得する
def get_artist(cells):
    artists = []
    member = list(cells[0].children)
    for i in range(len(member)//2 + 1):
        artists.append(member[i*2])
    detail = cells[3].text
    guests = ["戌亥とこ", "獅子神レオナ"]
    for guest in guests:
        if guest in detail:
            artists.append(guest)
    return artists

# video_idとt、アーティスト、曲名を取り出す
# 先頭だけ特殊なセルなためスキップする
tmp = []
for row in rows[1:]:
    cells = row.find_all('td')
    if check_cell(cells):
        videoid, time = get_videoid_time(cells)
        artists = get_artist(cells)
        artists_string = ','.join(artists)
        song_title = cells[2].text
        tmp.append([artists_string, song_title, videoid, time])
        
df = pd.DataFrame(data=np.array(tmp), 
                  columns=['Artists', 'SongTitle', 'VideoID', 'StartTime'])
df.to_csv('dataset/streams.csv', index=False)

In [ ]:
# あれ、out side frameがない？？？

# オリ曲を管理しているテーブル
# こちらもリンクがない、かつ数が少ないので手動で作成する方針とする
songs = soup.find_all("table", {"id":"content_block_26"})[0]
rows = songs.find_all("tr")

In [22]:
# 歌みた曲を管理しているテーブル
songs = soup.find_all("table", {"id":"content_block_34"})[0]
rows = songs.find_all("tr")

# 星街すいせいの曲だけを取得する
def check_cell(cells):
    text = cells[1].text
    if "星街すいせい" not in text:
        return False
    # コーラス参加などは除去する
    if "(星街すいせい)" in text:
        return False
    return True

# リンクから動画IDを取得する
def get_videoid(cells):
    href = cells[2].a["href"]
    if "youtube" in href:
        stratid = href.find("v=")+2
        endid = href.find("&t=")
        videoid = href[stratid:endid]
    if "youtu.be" in  href:
        stratid = href.find(".be")+4
        endid = href.find("?t=")
        videoid = href[stratid:endid]        
    return videoid
    
# メンバーと備考からアーティストを取得する
def get_artist(cells):
    artists = []
    member = list(cells[1].children)
    for i in range(math.ceil(len(member) / 2)):
        if '(' not in member[i*2] and ')' not in member[i*2]:
            artists.append(member[i*2])        
    # 見た限りではゲスト名しか入っていない  
    guest = list(cells[3].children)
    if len(guest) > 0:
        for i in range(math.ceil(len(guest) / 2)):
            if '(' not in guest[i*2] and ')' not in guest[i*2]:
                artists.append(guest[i*2])
    return artists

# video_idとt、アーティスト、曲名を取り出す
# 先頭だけ特殊なセルなためスキップする
tmp = []
for row in rows[1:]:
    cells = row.find_all('td')
    if check_cell(cells):
        videoid = get_videoid(cells)
        artists = get_artist(cells)
        artists_string = ','.join(artists)
        song_title = cells[2].text
        tmp.append([artists_string, song_title, videoid])
        
df = pd.DataFrame(data=np.array(tmp), 
                  columns=['Artists', 'SongTitle', 'VideoID'])
df.to_csv('dataset/covers.csv', index=False)

,Artists,SongTitle,VideoID
0,星街すいせい,メルティランドナイトメア,sm4KMKurd8
1,星街すいせい,ブルーバード,PerzMoTou-
2,星街すいせい,創聖のアクエリオン,_8eUrkYNnL
3,星街すいせい,melty world,9pgap38Os_
4,"星街すいせい,ルピナス",劣等上等,X1T9M4F8sK
5,星街すいせい,Shine!!,WLj0Rg_bcv
6,"星街すいせい,久遠藍,黒羽満月,ルピナス",バレンタイン・キッス,-uQ4bc7otx
7,星街すいせい,覚醒,dPTeBvVJ_x
8,"星街すいせい,出雲霞,堰代ミコ",コワレヤスキ,_PpTrPbRg0
9,"星街すいせい,出雲霞,堰代ミコ",エイリアンエイリアン,qcQ9k7A5TD
